# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [ ]:
# imports
import os
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [ ]:
#get API Key
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not(api_key and api_key.startswith('sk-proj-') and len(api_key)>10):
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    

In [ ]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

openai = OpenAI()
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


# set up environment

System prompt and user prompt

In [ ]:
system_prompt = "You are very clever helper helping a student for any of his queries. \
You are to be guiding and encouraging to the question, \
and always give example to allow student to better understand.\n"

In [ ]:
def user_prompt_for(question):
    user_prompt = f"You are understanding the question {question}"
    user_prompt += "please provide detail guide in markdown. \
That explain the answer to the question and \
Provide detailed example to enforce the learning. \n\n"
    return user_prompt

In [ ]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [ ]:
# Get gpt-4o-mini to answer, with streaming
def stream_GPT(question):
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt_for(question)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_GPT(question)

In [ ]:
# Get Llama 3.2 to answer

def stream_Ollama(question):
    stream = ollama_via_openai.chat.completions.create(
        model = MODEL_LLAMA,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt_for(question)}
          ],
        stream=True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_Ollama(question)